In [199]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, LSTM, CuDNNLSTM, Flatten, Dropout
from keras import optimizers
from keras.utils import to_categorical
import keras
from keras import backend as K



In [200]:
class trainerHelper():
    
    def __init__(self, data, labels, win_size, num_sensor_data, num_classes):
        self.data = data
        self.labels = labels
        self.win_size = win_size
        self.sensor_data = num_sensor_data
        self.num_classes = num_classes
        self.data_length = self.data.shape[0]
        print(self.data_length)
        self.start = 0
        
    def windows(self):
 
        while self.start + self.win_size < self.data_length:
            yield int(self.start), int(self.start + self.win_size)
            self.start += (self.win_size/2)
    
    def segment_data(self,lenght):
        self.start = 0
        segments = np.empty((0, self.win_size, lenght))
        labels= np.empty((0, self.num_classes))
        for (start, end) in self.windows():
            x = np.zeros((1, self.win_size,lenght))
            x[0,:] = self.data[start:end,:]
            label = self.labels[start:end,:]
            if(x.shape[1] == self.win_size):
                segments = np.vstack([segments,x])
                lb = np.zeros((1, self.num_classes))
                lb[0,:] = label[-1]
                labels = np.vstack([labels,lb])
        return segments, labels

In [201]:
data1= pd.read_csv("ADL1Opportunity_locomotion.csv",header=None)
data2= pd.read_csv("ADL2Opportunity_locomotion.csv",header=None)
data3= pd.read_csv("ADL3Opportunity_locomotion.csv",header=None)
data4= pd.read_csv("ADL4Opportunity_locomotion.csv",header=None)
data5= pd.read_csv("ADL5Opportunity_locomotion.csv",header=None)



In [202]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def prepare_data(data):
    encoder = OneHotEncoder()
    print('Processing..')
    one_hot_labels = encoder.fit_transform(data['labels'].values.reshape(-1,1)).toarray()
    one_hot_labels.shape
    scaler = MinMaxScaler()
    data.drop(['labels'], 1, inplace=True)
    data.shape
    data = scaler.fit_transform(data)
    trainer_helper = trainerHelper(data, one_hot_labels, 15, data.shape[1]-1, 5)
    segments, labels = trainer_helper.segment_data(data.shape[1])

    return segments,labels

In [203]:
train_list=[];
test_list=[];
data1.rename(columns ={data1.shape[1]-1: 'labels'}, inplace =True)
[segments,labels1]=prepare_data(data1)
train_list.append([segments,labels1])

data2.rename(columns ={data2.shape[1]-1: 'labels'}, inplace =True)
[segments,labels2]=prepare_data(data2)
train_list.append([segments,labels2])


data3.rename(columns ={data3.shape[1]-1: 'labels'}, inplace =True)
[segments,labels3]=prepare_data(data3)
train_list.append([segments,labels3])

data4.rename(columns ={data4.shape[1]-1: 'labels'}, inplace =True)
[segments,labels4]=prepare_data(data4)
test_list.append([segments,labels4])

data5.rename(columns ={data5.shape[1]-1: 'labels'}, inplace =True)
[segments,labels5]=prepare_data(data5)
test_list.append([segments,labels5])

Processing..
51088
Processing..
32223
Processing..
33274
Processing..
32955
Processing..
32955


In [204]:
#COrrect reshape 

#Train
for train_instance in train_list:
    train_instance[0]=train_instance[0].transpose(0,2,1)
#TEst
for test_instance in test_list:
    test_instance[0]=test_instance[0].transpose(0,2,1)

In [205]:
K.clear_session()
model= Sequential()


win_size = 15
classes = 5
num_sensors = data1.shape[1]
kernel_height = 5
inputshape = (num_sensors, win_size)
trainSplitRatio=0.8;




In [206]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu',
                 input_shape=inputshape))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Dropout(0.5))

model.add(LSTM(128, return_sequences=True))

model.add(Dropout(0.5))

model.add(LSTM(128, return_sequences=False))

model.add(Dropout(0.5))

model.add(Dense(classes, activation='softmax'))

adam = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [207]:
for layer in model.layers:
    print(layer.name)

conv1d_1
conv1d_2
conv1d_3
conv1d_4
dropout_1
lstm_1
dropout_2
lstm_2
dropout_3
dense_1


In [208]:
batchSize = 100
# TRAIN
trainX=[]
trainY=[]
for train_instance in train_list:
    trainX.append(train_instance[0])
    trainY.append(train_instance[1])

trainX=np.concatenate(trainX,axis=0)
trainY=np.concatenate(trainY,axis=0)    

print(trainX.shape)
print(trainY.shape)

#TEST

testX=[]
testY=[]
for test_instance in test_list:
    testX.append(test_instance[0])
    testY.append(test_instance[1])

testX=np.concatenate(testX,axis=0)
testY=np.concatenate(testY,axis=0) 




model.fit(trainX,trainY,epochs=1,batch_size=batchSize,verbose=1)
print('Calculating score.. ')
score = model.evaluate(testX,testY,verbose=1)
print(score)
model.save('CIP_ADL_CNN4_LSTM2_model.h5')

(15540, 113, 15)
(15540, 5)
Epoch 1/1
8784/8784 [==============================] - 27s 3ms/step
[1.4638833248332985, 0.34631147540983609]


In [197]:
predictions = model.predict(testX)

In [198]:
count = 0.0
den = 0.0
for pair in zip(predictions, testY):
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    den += 1.0

print(count / den)

0.3185336976320583


In [38]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [39]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)